# Interfaces

Every class has an interface: with interface we mean the set of public attributes (methods and data attributes) implemented or inherited by the class. <br>
By definition protected and private attributes are not part of an interface.

In [ ]:
# Example: x and y as public data attributes

class Vector2d:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __iter__(self):
        return (i for i in (self.x, self.y))

We then turned x and y into read only properties

In [ ]:
class Vector2d:
    typecode = d

    def __init__(self, x, y):
        self.__x = x
        self.__y = y

    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    def __iter__(self):
        return (i for i in (self.x, self.y))

Another useful definition for interface is: the subset of an object's public method that allow the object to play a spacific role in the system.

### Python digs sequences (or classes implementing the protocol)

In [1]:
# Example: partial sequence protocol implementation

class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]
    
f = Foo()
print(f[1])
for i in f: print(i)
print(20 in f)

10
0
10
20
True


There is no `__iter__` method, but python whenever sees `__getitem__` tries to iterate over the object by calling the `__getitem__` method. Same goes for the `__contains__` method. <br>
Python does this because of the importance of the sequence protocol.

In [6]:
# Example: a deck as a sequence of cards

from collections import namedtuple

Card = namedtuple('Card', ['rank', 'suit'])

class Deck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades hearts clubs diamonds'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for rank in self.ranks for suit in self.suits]

    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, index):
        return self._cards[index]
    

In [7]:
# the shuffle PSL method does not work as is

from random import shuffle

deck = Deck()
shuffle(deck)

TypeError: 'Deck' object does not support item assignment

But mutable sequences also need to provide the `__setitem__` method, so far our deck only implements the immutable sequence protocol.

In [ ]:
# Example of monkey patching
from random import shuffle

def set_card(deck: Deck, position: int, card: Card):
    deck._cards[position] = card

Deck.__setitem__ = set_card

deck = Deck()

shuffle(deck)

This is an example of monkey patching: changing a class or module at runtime without modifying the source code.

## ABCs

You can subclass classes from abstract classes or register classes to them. Sometimes there is not even the need to register them, that is the case for some simple abstract classes.

In [1]:
class Struggle:
    def __len__(self):
        return 23
    
from collections import abc
isinstance(Struggle(), abc.Sized)

True

`isinstance` starts making sense when we use it to compare objects with abstract metaclasses. Otherwise it would be just a limitation on duck typing.

In [ ]:
# Example: duck typing to handle a string or an iterable of strings

try:
    field_names = field_names.replace(',', ' ').split()
except AttributeError:
    pass
field_names = tuple(field_names)

## Subclassing an ABC

In [ ]:
import collections

type(collections.abc.MutableSequence)

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades clubs hearts diamonds'

    def __init__(self):
        self._cards = [Card(rank, suit) for rank in self.ranks for suit in self.suits]

    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, index):
        return self._cards[index]
    
    def __setitem__(self, index, card):
        self._cards[index] = card

    def __delitem__(self, index):
        del(self._cards[index])
    
    def insert(self, index, card):
        self._cards.insert(index, card)

abc.ABCMeta

Python checks for the implementation of abstract methods only at runtime.

## ABCs in the standard library

## Defining and using an ABC

In [2]:
import abc

class Tombola(abc.ABC):

    @abc.abstractmethod
    def load(self, iterable):
        """Add items from an iterable."""

    @abc.abstractmethod
    def pick(self):
        """Remove item at random, returning it."""

    def loaded(self):
        return bool(self.inspect())
    
    def inspect(self):
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))


In [2]:
# Implement a fake tombola

class Fake(Tombola):
    def pick(self):
        return 13

print(Fake)
f = Fake()

<class '__main__.Fake'>


TypeError: Can't instantiate abstract class Fake without an implementation for abstract method 'load'

### ABC syntax details
Abstract classmethods and abstract properties are achieved stacking decorators on top of each others.

### Subclassing ABCs

In [3]:
# Subclassing Tombola ABC

import random

class BingoCage(Tombola):

    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
    
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    
    def __call__(self):
        self.pick()

In [4]:
# Subclassing Tombola again

class LotteryBlower(Tombola):

    def __init__(self, iterable):
        self._balls = list(iterable)

    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('Pick from empty LotteryBlower')
        return self._balls.pop(position)
    
    def loaded(self):
        return bool(self._balls)
    
    def inspect(self):
        return tuple(sorted(self._balls))

## Virtual subclass of ABCs
An essential feature of goose typing is the possibility to dynamically register a classas as a virtual subclass of an ABC even if it does not inherit from it. <br>
When we register a class in this way python believes us withoud checking <br>
The virtual subclass does not inherit from the abstract class. <br>
The virtual class can be registered by calling a plain function or by using a decorator.

In [3]:
# Tombolist a real subclass of list, a virtual subclass of Tombola

from random import randrange

@Tombola.register # or Tombola.register(Tombolist)
class TomboList(list):

    def pick(self):
        if self:
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')
    
    load = list.extend

    def loaded(self):
        return bool(self)
    
    def inspect(self):
        return tuple(sorted(self))
    

print(issubclass(TomboList, Tombola))
t = TomboList(range(100))
print(isinstance(t, Tombola))

True
True


Inheritance is guided by the class attribute `__mro__` it only shows the "real" superclasses

In [4]:
TomboList.__mro__

(__main__.TomboList, list, object)

## ABCs magic, geese can behave as ducks
A class can be recognized as a virtual subclass of an ABC even without registration. This depends with which abstract class we are comparing our class with (for example `Sized`). The abstract classes in question need to provide a `__subclasshook__` method.

In [5]:
class Struggle:
    def __len__(self): return 23

from collections import abc
print(isinstance(Struggle(), abc.Sized))
print(issubclass(Struggle, abc.Sized))

True
True
